# Import libraries and data

In [1]:
#import libraries
import pandas as pd
import pyarrow
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
#display max 500 columns of dfs for the whole notebook
pd.set_option('display.max_columns', 500)
#display two decimals number for all float data for the whole notebook
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
#merge all files ending with parquet in "C:/Users/Anaïs WERNLE/Desktop/NYC trips/yellow" and store data in yellowdf_merged
yellow = glob.glob("C:/Users/Anaïs WERNLE/Desktop/NYC trips/yellow/*.parquet")
yellowdf_merged= pd.concat([pd.read_parquet(f) for f in yellow])

# Data cleaning : identify and delete anomalies
### ex : ride that last longer than a full day, more than 4 passagers in a taxi ... 

In [4]:
#look for anomalies such as empty columns, mins that are too low or max that are too high
yellowdf_merged.describe(include='all')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,30904308.00,30904308,30904308,29425613.00,30904308.00,29425613.00,29425613,30904308.00,30904308.00,30904308.00,30904308.00,30904308.00,30904308.00,30904308.00,30904308.00,30904308.00,30904308.00,29425613.00,25262890.00
unique,NaN,16906139,16893760,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,2021-09-15 17:22:00,2021-12-08 00:00:00,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,28,39,NaN,NaN,NaN,29005614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,2002-12-31 23:07:20,2003-01-01 00:38:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2098-09-11 02:23:31,2098-09-11 02:52:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.71,NaN,NaN,1.43,6.92,1.09,NaN,164.93,162.34,1.19,13.52,1.05,0.49,2.34,0.39,0.30,19.70,2.29,0.07
std,0.53,NaN,NaN,1.03,698.38,1.97,NaN,66.40,70.81,0.53,178.99,1.25,0.08,2.83,1.81,0.04,179.19,0.73,0.29
min,1.00,NaN,NaN,0.00,0.00,1.00,NaN,1.00,1.00,0.00,-758.00,-5.50,-0.55,-333.32,-88.75,-0.30,-951.00,-2.50,-1.25
25%,1.00,NaN,NaN,1.00,1.10,1.00,NaN,130.00,112.00,1.00,6.50,0.00,0.50,0.00,0.00,0.30,11.76,2.50,0.00


In [5]:
#change tpep_pickup_datetime and tpep_dropoff_datetime to dateformat (which is necessary to filter the df as written below 
yellowdf_merged['tpep_pickup_datetime'] = pd.to_datetime(yellowdf_merged['tpep_pickup_datetime'], format='%YYYY-%MM-%DD %H:%M:%S')
yellowdf_merged['tpep_dropoff_datetime'] = pd.to_datetime(yellowdf_merged['tpep_dropoff_datetime'], format='%YYYY-%MM-%DD %H:%M:%S')

#delete all rows of df where pickup happens after dropoff
yellowdf_merged = yellowdf_merged[yellowdf_merged['tpep_pickup_datetime'] < yellowdf_merged['tpep_dropoff_datetime']]

#delete all data that are not from 2021 
yellowdf_merged = yellowdf_merged[(yellowdf_merged['tpep_dropoff_datetime'].dt.year == 2021)]

In [6]:
#add time in new columns "pick_time" and "drop off" 
yellowdf_merged["pickup_time"] = yellowdf_merged['tpep_pickup_datetime'].dt.time
yellowdf_merged["dropoff_time"] = yellowdf_merged['tpep_dropoff_datetime'].dt.time

#assign the hour format to those columns
yellowdf_merged['dropoff_time'] = pd.to_datetime(yellowdf_merged['dropoff_time'], format='%H:%M:%S')
yellowdf_merged['pickup_time'] = pd.to_datetime(yellowdf_merged['pickup_time'], format='%H:%M:%S')

#calculate time between pick-up and drop-off
yellowdf_merged['trip_duration_min'] = yellowdf_merged['tpep_dropoff_datetime'].sub(yellowdf_merged['tpep_pickup_datetime']).dt.total_seconds().div(60)

In [7]:
#delete all rows with a ride that lasts more than 4 hours
yellowdf_merged = yellowdf_merged[(yellowdf_merged['trip_duration_min'] < 240) & (yellowdf_merged['trip_duration_min'] >= 1)] 

#delete all rows with a ride that lasts more than 4 hours that cost less than $2.50 which is the miminum price in NY
yellowdf_merged = yellowdf_merged[(yellowdf_merged['total_amount'] >= 2.50)]

#delete all rows with a ride with trip_distance > 50km. 
yellowdf_merged = yellowdf_merged[(yellowdf_merged['trip_distance'] <= 50)]

#delete all row with location_id corresponding to uknown area (location_id = 264 or 265)
yellowdf_merged = yellowdf_merged[(yellowdf_merged['PULocationID'] != 265) & (yellowdf_merged['PULocationID'] != 264)]
yellowdf_merged = yellowdf_merged[(yellowdf_merged['DOLocationID'] != 265) & (yellowdf_merged['DOLocationID'] != 264)]

#calculate the price by minute of each ride
yellowdf_merged["price_by_min"] = (yellowdf_merged['total_amount']/yellowdf_merged['trip_duration_min'])

#delete all rows with price by min higher than $5 and lower than $0
#the median price of all rides is $1.45 by minute (to calculate it : median_price_minutes = (greendf_merged['total_amount']/greendf_merged['trip_duration_min']).median())
yellowdf_merged = yellowdf_merged[(yellowdf_merged["price_by_min"] < 5) & (yellowdf_merged["price_by_min"] > 0)]

#calculate average speed of each rides
yellowdf_merged['avg_speed_miles'] = yellowdf_merged["trip_distance"]/(yellowdf_merged['trip_duration_min']/60)

#delete all rows with average speed higher than 70 mph (as the limitation in NY state is 55 mph. 
yellowdf_merged = yellowdf_merged[(yellowdf_merged["avg_speed_miles"] < 100)]

#delete all rows with pasenger count = 0 or is na
yellowdf_merged = yellowdf_merged[(yellowdf_merged['passenger_count'] > 0) & (yellowdf_merged['passenger_count'] < 5)]
yellowdf_merged = yellowdf_merged.dropna(subset=['passenger_count'])

In [8]:
yellowdf_merged.describe(include = 'all')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_time,dropoff_time,trip_duration_min,price_by_min,avg_speed_miles
count,26631790.00,26631790,26631790,26631790.00,26631790.00,26631790.00,26631790,26631790.00,26631790.00,26631790.00,26631790.00,26631790.00,26631790.00,26631790.00,26631790.00,26631790.00,26631790.00,26631790.00,22871459.00,26631790,26631790,26631790.00,26631790.00,26631790.00
unique,NaN,15657545,15653590,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86400,86400,NaN,NaN,NaN
top,NaN,2021-10-21 13:57:45,2021-12-01 18:30:31,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01 18:10:02,1900-01-01 00:00:00,NaN,NaN,NaN
freq,NaN,12,13,NaN,NaN,NaN,26263162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,628,837,NaN,NaN,NaN
first,NaN,2020-12-31 23:55:40,2021-01-01 00:02:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01 00:00:00,1900-01-01 00:00:00,NaN,NaN,NaN
last,NaN,2021-12-31 23:56:49,2021-12-31 23:59:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01 23:59:59,1900-01-01 23:59:59,NaN,NaN,NaN
mean,1.69,NaN,NaN,1.30,3.03,1.07,NaN,164.67,162.36,1.23,12.71,1.06,0.50,2.34,0.34,0.30,18.91,2.33,0.07,NaN,NaN,13.82,1.60,12.11
std,0.46,NaN,NaN,0.63,3.73,1.95,NaN,65.30,70.14,0.44,10.42,1.24,0.03,2.53,1.55,0.00,13.13,0.63,0.29,NaN,NaN,10.81,0.61,6.12
min,1.00,NaN,NaN,1.00,0.00,1.00,NaN,1.00,1.00,1.00,-0.80,0.00,0.00,0.00,0.00,0.00,2.50,0.00,0.00,NaN,NaN,1.00,0.01,0.00
25%,1.00,NaN,NaN,1.00,1.10,1.00,NaN,132.00,113.00,1.00,6.50,0.00,0.50,0.00,0.00,0.30,11.76,2.50,0.00,NaN,NaN,6.80,1.18,8.29


# Standardization : unify column names and change to the right data types 

### ex : delete all capital letters and spaces in columns names, change ID columns to string format

In [9]:
#display column names 
yellowdf_merged.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'pickup_time',
       'dropoff_time', 'trip_duration_min', 'price_by_min', 'avg_speed_miles'],
      dtype='object')

In [10]:
#lower the case of all column names
yellowdf_merged.columns= yellowdf_merged.columns.str.lower()
#change column names
yellowdf_merged.rename(columns = {'pulocationid':'pickup_location_id', 'dolocationid':'dropoff_location_id', 'vendorid' : 'vendor_id', 'ratecodeid' : 'rate_code_id'}, inplace = True)
yellowdf_merged= yellowdf_merged.drop([ "vendor_id", "store_and_fwd_flag", "payment_type", 
                                       "improvement_surcharge", "tolls_amount", "fare_amount", "extra", "congestion_surcharge", "rate_code_id", "mta_tax"], axis=1)

In [11]:
#verify all data types
yellowdf_merged.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
pickup_location_id                int64
dropoff_location_id               int64
tip_amount                      float64
total_amount                    float64
airport_fee                     float64
pickup_time              datetime64[ns]
dropoff_time             datetime64[ns]
trip_duration_min               float64
price_by_min                    float64
avg_speed_miles                 float64
dtype: object

In [12]:
#change data types
yellowdf_merged["passenger_count"] = yellowdf_merged["passenger_count"].astype(int)
yellowdf_merged["pickup_location_id"] = yellowdf_merged["pickup_location_id"].astype(str)
yellowdf_merged["dropoff_location_id"] = yellowdf_merged["dropoff_location_id"].astype(str)

In [13]:
#export data
yellowdf_merged.to_parquet('yellow.gzip', compression='gzip')